In [47]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
import os
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed
from sklearn.base import clone


In [48]:
#modele takie same jak w oryginale, wyrzuciłem jedynie kryterium podziału bo w regresji to jest po prostu mse

########################################### MODELS ##########################################
cpu_no = 12
rf_100 =    ('RF', RandomForestRegressor(random_state=123, n_jobs=cpu_no, n_estimators=100))
rf_200 =    ('RF[200]', RandomForestRegressor(random_state=123, n_jobs=cpu_no,  n_estimators=200))
rf_500 =    ('RF[500]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, n_estimators=500))
#rf_entr =   ('  RF[entr]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, criterion="entropy"))
rf_md_10 =  ('RF[md10]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_depth=10))
rf_md_15 =  ('RF[md15]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_depth=15))
rf_md_20 =  ('RF[md20]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_depth=20))
rf_md_25 =  ('RF[md25]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_depth=25))
rf_mss_3 =  ('RF[mss3]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_split=3))
rf_mss_4 =  ('RF[mss4]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_split=4))
rf_mss_6 =  ('RF[mss6]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_split=6))
rf_mss_8 =  ('RF[mss8]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_split=8))
rf_msl_2 =  ('RF[msl2]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_leaf=2))
rf_msl_3 =  ('RF[msl3]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_leaf=3))
rf_msl_4 =  ('RF[msl4]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_leaf=4))
rf_msl_5 =  ('RF[msl5]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, min_samples_leaf=5))
rf_mf_log = ('RF[mfLog2]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_features="log2"))
rf_mf_all = ('RF[mfAll]', RandomForestRegressor(random_state=123, n_jobs=cpu_no, max_features=None))
######################################## END OF MODELS ######################################

######################################### PARAMETERS ########################################
max_samples = [0.2, 0.6, 0.8, 1, 1.2, 2, 3, 4, 5]
forests = [rf_100, rf_200, rf_500, rf_md_10, rf_md_15, rf_md_20, rf_md_25, rf_mss_3, rf_mss_4, rf_mss_6, rf_mss_8,
        rf_msl_2, rf_msl_3, rf_msl_4, rf_msl_5, rf_mf_log, rf_mf_all]
##################################### END OF PARAMETERS #####################################

#duplikujemy dane zeby dalo sie zrobic br>1
def make_X_train(X_tra, y_tra, max_sample):
    if max_sample <= 1:
        return X_tra, y_tra
    elif 1 < max_sample < 2:
        return np.tile(X_tra, (2, 1)), np.tile(y_tra, 2)
    else:
        return np.tile(X_tra, (max_sample, 1)), np.tile(y_tra, max_sample)

In [49]:
# os.getcwd()
# os.chdir('pi_br_greater_than_1')

In [50]:
#to zrobcie tak jak tutaj albo wpiszcie ręcznie do dataset_paths sciezki do datasetow

path_to_datasets = './test_data' #'path/to/folder/with/datasets'

dataset_paths = []
for filename in os.listdir(path_to_datasets):
    full_path = os.path.join(path_to_datasets, filename)
    dataset_paths.append(full_path)

In [51]:
def process_fold(train_index, test_index, X, y, ms, rfr):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_tr, y_tr = make_X_train(X_train, y_train, ms)

    rfr_fold = clone(rfr[1])
    rfr_fold.max_samples = None if ms >= 1 and ms != 1.2 else (0.6 if ms == 1.2 else ms)
    rfr_fold.fit(X_tr, y_tr)

    y_hat = rfr_fold.predict(X_test)
    return mean_squared_error(y_test, y_hat)

In [ ]:
results = {'dataset' : [],
           'bootstrap_rate': [],
           'rf' : [],
           'cv_mse_scores' : [],
           'mean_mse' : [],
           'std_mse' : []}

scaler = StandardScaler()
cvsplit = RepeatedKFold(n_splits=2, n_repeats=200, random_state=42)

for dataset_path in dataset_paths:
    df = pd.read_csv(dataset_path)

    X = df.iloc[:, :-1].to_numpy()
    y = df.iloc[:, -1].to_numpy()

    X = scaler.fit_transform(X)

    for ms in max_samples:
        for rfr in forests:
            print(f'Processing {dataset_path} with bootstrap rate {ms} and model {rfr[0]}  ...')

            cv_mse_scores = Parallel(n_jobs=-1)(
                delayed(process_fold)(train_index, test_index, X, y, ms, rfr)
                for train_index, test_index in cvsplit.split(X, y)
            )

            results['dataset'].append(dataset_path)
            results['bootstrap_rate'].append(ms)
            results['rf'].append(rfr[0])
            results['cv_mse_scores'].append(cv_mse_scores)
            results['mean_mse'].append(f'{np.mean(cv_mse_scores):.3f}')
            results['std_mse'].append(f'{np.std(cv_mse_scores):.3f}')


Processing ./test_data\Abalone_1_preprocessed.csv with bootstrap rate 0.2 and model RF...
<generator object _RepeatedSplits.split at 0x000001CEDB010500>


KeyboardInterrupt: 

In [43]:
wyniki = pd.DataFrame(results)
wyniki.to_csv('wyniki_bb.csv', index=False)

In [46]:
res = pd.read_csv('wyniki_bb.csv')
res

,dataset,bootstrap_rate,rf,cv_mse_scores,mean_mse,std_mse
0,./test_data\Abalone_1_preprocessed.csv,0.2,RF,"[4.670553470560077, 4.685861925287356, 4.42996...",4.653,0.209
1,./test_data\Abalone_1_preprocessed.csv,0.2,RF[200],"[4.670646110579225, 4.612593390804598, 4.40546...",4.641,0.209
2,./test_data\Abalone_1_preprocessed.csv,0.2,RF[500],"[4.675499676400192, 4.5899594367816094, 4.3954...",4.623,0.215
3,./test_data\Abalone_1_preprocessed.csv,0.2,RF[md10],"[4.666371538623958, 4.648686912704007, 4.42591...",4.642,0.218
4,./test_data\Abalone_1_preprocessed.csv,0.2,RF[md15],"[4.67435557548462, 4.670776546694385, 4.421879...",4.650,0.212
...,...,...,...,...,...,...
148,./test_data\Abalone_1_preprocessed.csv,5.0,RF[msl3],"[5.479795979705167, 5.218875207540328, 5.10822...",5.304,0.200
149,./test_data\Abalone_1_preprocessed.csv,5.0,RF[msl4],"[5.443587728962996, 5.119611281061154, 5.03928...",5.239,0.213
150,./test_data\Abalone_1_preprocessed.csv,5.0,RF[msl5],"[5.4389756142294114, 5.077534386478442, 5.0209...",5.214,0.217
151,./test_data\Abalone_1_preprocessed.csv,5.0,RF[mfLog2],"[4.989540689325036, 4.857929070881227, 4.77820...",4.885,0.207
